# **PLN - Análise de Sentimentos de Tweets**

Aquisição dos dados

Para essa etapa irei utilizar o dataset **Tweets_Msg.csv**

### Importando Bibliotecas

In [1]:
import nltk
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, log_loss

### Importando Dataset

In [2]:
ds_Tweets = pd.read_csv('Tweets_Msg.csv',encoding='utf-8')
ds_Tweets.head(10)

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,Observação,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",NaN,NaN,Um Outro Mundo É Possível,Maristela Guimarães,florpimenta,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Wed Jan 04 23:00:53 +0000 2017,"""..E 25% dos mineiros dizem não torcer para ti...",NaN,NaN,NaN,Adriano,almanakfc,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Sun Jan 08 18:34:22 +0000 2017,"""A gigantesca barba do mal"" em destaque no cad...",NaN,NaN,"Belo Horizonte, Brasil",Editora Nemo,editoranemo,2,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Wed Jan 04 22:55:08 +0000 2017,"""BB e governo de Minas travam disputa sobre de...",NaN,NaN,NaN,Minas Previ,MinasPrevi,0,Negativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Sat Jan 07 01:37:10 +0000 2017,"""com vcs bh fica pequena!"" Belo Horizonte (pro...",NaN,NaN,Belo Horizonte - MG,Marina.,mmarinaey,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Informações do dataset

In [3]:
ds_Tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8199 entries, 0 to 8198
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 8199 non-null   int64  
 1   Created At                 8199 non-null   object 
 2   Text                       8199 non-null   object 
 3   Geo Coordinates.latitude   104 non-null    float64
 4   Geo Coordinates.longitude  104 non-null    float64
 5   User Location              5489 non-null   object 
 6   Username                   8199 non-null   object 
 7   User Screen Name           8199 non-null   object 
 8   Retweet Count              8199 non-null   int64  
 9   Classificacao              8199 non-null   object 
 10  Observação                 1 non-null      object 
 11  Unnamed: 10                0 non-null      float64
 12  Unnamed: 11                0 non-null      float64
 13  Unnamed: 12                0 non-null      float

Verificado que o dataset possi 8199 tweets e que a coluna alvo "Classificacao" não possui valores nulos

### Avaliação dos dados de Classificação

In [4]:
# Criando um dataset só com a classificação para agrupar
ds_Tweets_Agrupado = ds_Tweets.groupby(['Classificacao']).count()
ds_Tweets_Agrupado.rename(columns={'Unnamed: 0': 'Total'}, inplace = True)
ds_Tweets_Agrupado

,Total,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Observação,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
Classificacao,,,,,,,,,,,,,,,,,,,,,,,,,
Negativo,2446,2446,2446,1,1,1659,2446,2446,2446,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Neutro,2453,2453,2453,102,102,1712,2453,2453,2453,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Positivo,3300,3300,3300,1,1,2118,3300,3300,3300,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Excluindo Variáveis
ds_Tweets_Agrupado = pd.DataFrame(ds_Tweets_Agrupado[['Total']])
ds_Tweets_Agrupado

,Total
Classificacao,
Negativo,2446
Neutro,2453
Positivo,3300


In [6]:
# Copiando Dataframe
ds_Tweets_Novo = ds_Tweets.copy()

## Pré-Processamento dos dados

In [7]:
# Deixando as frases em palavras minúsculas e remover caracteres especiais
for i in range(len(ds_Tweets)):
    ds_Tweets_Novo.Text[i] = ds_Tweets['Text'][i].lower()
    ds_Tweets_Novo.Text[i] = re.sub(r'[^a-zA-Z\s]', '', ds_Tweets_Novo['Text'][i], re.I|re.A)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# Irei separar os tweets e suas classificações
tweets = ds_Tweets_Novo["Text"].values
classificacao = ds_Tweets_Novo['Classificacao'].values

In [116]:
tweets.shape

(8199,)

In [117]:
classificacao.shape

(8199,)

# **Treinamento do Modelo**

Para esta abordagem irei utilizar o modelo Bag of Words e o algoritmo Naive Bayes Multinomial

In [9]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)

In [10]:
modelo = MultinomialNB()
modelo.fit(freq_tweets, classificacao)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
# Usando frases de teste para fazer a classificação com o modelo treinado
testes = ["Esse governo está no início, vamos ver o que vai dar",
          "Estou muito feliz com o governo de São Paulo esse ano",
          "O estado de Minas Gerais decretou calamidade financeira!!!",
          "A segurança desse país está deixando a desejar",
          "O governador de Minas é do PT",
          "O prefeito de São Paulo está fazendo um ótimo trabalho"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro', 'Positivo'],
      dtype='<U8')

# **Avaliação do Modelo**

In [13]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_tweets, classificacao, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

## Métricas do Modelo

In [14]:
metrics.accuracy_score(classificacao, resultados)

0.8755945847054519

In [42]:
# Medidas de validação do modelo
sentimentos = ["Positivo", "Negativo"]
print(metrics.classification_report(classificacao, resultados))

              precision    recall  f1-score   support

    Negativo       0.88      0.92      0.90      2446
      Neutro       0.78      0.84      0.81      2453
    Positivo       0.95      0.87      0.91      3300

    accuracy                           0.88      8199
   macro avg       0.87      0.88      0.87      8199
weighted avg       0.88      0.88      0.88      8199



In [43]:
# Gerando a Matriz de Confusão
print(pd.crosstab(classificacao, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2254     183         9  2446
Neutro         252    2057       144  2453
Positivo        45     387      2868  3300
All           2551    2627      3021  8199
